# Examples of higher order functions and partial application

_Note_: ensure that students copy, by hand and on paper, the various definitions written by the teacher on the whiteboard. It is strongly advised to ask students *not* to use a laptop, as it will prove distracting.

The topics discussed are:

- let us test the limits of the theory we have just given by building simple but concrete programs with it
- we define some functions by only using the generic combinators `repeat` and `then`
    - as usual, let us begin with a very simple problem: drawing lines recursively

In [2]:
def line(n):
    if n <= 0:
        return ""
    else:
        return "*" + line(n-1)
line(int(input()))

3


'***'

- if we observe the inductive case, we could read the expression `"*" + line(n-1)` as _draw an asterisk, then the rest of the line_
- we might want to rewrite the code so that it more literally matches the description:
    - our target is the highly descriptive `then(asterisk, line(n-1))` (_prefix_ instead of _infix_, but it is the same)
- let us reformulate the task of `line`
    - instead of just producing the line of asterisks, we want to **create a function that will add a line of asterisks to some initial string**
    - so `line` returns a function that, given a string, adds a line of asterisks, for example:
    - in general, drawing anything will become a function that adds the drawn element to a string which is passed as a parameter
    
- according to our new formulation of drawing:
    - drawing nothing will simply add nothing to the string it receives as input
    - drawing an asterisk will simply add an asterisk to the string it receives as input
    - drawing a newline will simply add a newline to the string it receives as input    
    - ..

In [24]:
def nothing(s): return s
def asterisk(s): return s + "*"
def space(s): return s + " "
def newline(s): return s + "\n"

print(nothing(""))
print(nothing("..."))

print(asterisk(""))
print(asterisk("..."))


...
*
...*


- suppose we wanted to draw more than an asterisk
- we would simply call the asterisk function multiple times:

In [19]:
print(asterisk(asterisk("")))
print(asterisk(asterisk(asterisk(""))))

**
***


- because of the many brackets, this is a bit cumbersome to read
- if we observe the chain of calls to `asterisk` though, we could observe that we are simply calling `asterisk` on the output of another `asterisk` function (or more)
- we could represent this explicitly by using the `then` function, which is exactly built to encapsulate this concept

In [25]:
def then(f,g): 
    return lambda x: g(f(x))

print(then(asterisk, asterisk)(""))
print(then(asterisk, then(asterisk, asterisk))(""))
print(then(asterisk, then(asterisk, asterisk))("!!!"))
print(then(asterisk, then(space, asterisk))(""))

**
***
!!!***
* *


- we are now building pipelines of drawing functions, but until we pass the starting string, nothing gets drawn
    - we are delaying drawing now
- of course we could also notice that `nothing` can always be added to such a pipeline without changing its effect

In [5]:
print(then(nothing, asterisk)(""))
print(then(asterisk, nothing)(""))

*
*


- at this point, we can easily (for a large enough value of "easily") assemble the `line` function
- an empty line simply _adds nothing_, as there is nothing to add
- a non-empty line first adds an asterisk, then the rest of the line

In [6]:
def line(n):
    if n <= 0:
        return nothing
    else:
        return then(asterisk, line(n-1))

line(int(input()))("")

3


'***'

- let us continue our journey
- a rectangle with `n` rows and `m` columns is drawn as:
    - nothing if `n` is zero
    - a line, then a newline, and then a rectangle with n-1 rows otherwise
- in code, this simply becomes:

In [8]:
def rectangle(n,m):
    if n <= 0:
        return nothing
    else:
        return then(then(line(m), newline), rectangle(n-1,m))
    
n = int(input())
print(rectangle(n,n)(""))

3
***
***
***



- of course the very same treatment can be applied to all functions we have seen so far
    - including general-looking functions such as `repeat`
    - we could reformulate `repeat` as a function which accepts a function as input, and then executes it `n` times starting from the same input
    - `repeat(f,n)` will thus yield $\texttt{lambda x:} \underbrace{\texttt{f(...(f(x))...)}}_{n \text{ times}}$
- we can implement `repeat` easily as a function which 
    - returns `nothing` if `n` is zero, as there is nothing to do
    - first performs `f` once, `then` it `repeat`s `f` `n-1` times otherwise 

In [18]:
def repeat(f,n):
    if n <= 0:
        return nothing
    else:
        return then(f, repeat(f, n-1))

print(repeat(lambda x: x * 2, int(input()))(1))

10
1024


- notice how now functions such as `line` recover their original structure based upon `repeat`, just at another (higher) level of abstraction

In [21]:
def line(n): return repeat(asterisk, n)

print(line(int(input()))(""))

5
*****


- similarly, a square will also become the repetition of lines (each followed by a newline of course):

In [23]:
def square(n): return repeat(then(line(n), newline), n)

print(square(int(input()))(""))

5
*****
*****
*****
*****
*****



- hollow squares are built up from hollow lines, but for the rest follow the same approach:

In [36]:
def hollow_line(n): return then(asterisk, then(repeat(space, n-2), asterisk))
def hollow_square(n): return then(then(line(n), newline), then(repeat(then(hollow_line(n), newline), n-2), line(n)))

print(hollow_square(int(input()))(""))

4
****
*  *
*  *
****
